In [1]:
# test full training of maskcnn_polished_with_rcnn_k_bl
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict

from thesis_v2.data.prepared.cadena_plos_cb19 import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl,
    load_modules
)

In [2]:
def train_one_maskcnn_polished_with_rcnn_k_bl(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    n_timesteps,
):
    
    load_modules()
    datasets = get_data(
         px_kept=80, final_size=input_size, scale=0.5,
        seed=split_seed
    )

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_rcnn_k_bl(
                                    input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    n_timesteps=n_timesteps,
                                    factored_constraint=None,
                                    blstack_pool_ksize=1,
                                    blstack_pool_type=None,
                                    acc_mode='cummean',
                                    bn_after_fc=False,
                                    ff_1st_block=True,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/test_full_training_maskcnn_polished_with_rcnn_k_bl/ff_1st_block_cb19_data_cls4/{model_seed}',
        show_every=100,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params={
            # reduce on batch axis
            'eval_fn': {
                'yhat_reduce_axis': 1,
            }
        },
        print_model=True
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 3,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 0,
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'loss_type': 'mse',
        'smoothness': 0.000005,

        
        'smoothness_name': '0.000005',
        

        
        'scale': 0.01,
        'scale_name': '0.01',
        'input_size': 40,
        'n_timesteps': 4,
    }
}

print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular))

# 27629 = 1*2 + 16*1*9*9 + 16*2 + 2*(16*16*3*3+16*16*3*3+4*16*2) + 79*(14*14 + 16 + 1) 

{'fc', 'final_act', 'bn_output', 'pooling'}
['conv0', 'bl_stack.layer_list.0.b_conv', 'bl_stack.layer_list.1.b_conv']
num_param 26672
JSONNet(
  (moduledict): ModuleDict(
    (accumulator): RecurrentAccumulator()
    (act0): ReLU()
    (bl_stack): BLConvLayerStack(
      (layer_list): ModuleList(
        (0): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (l_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (l_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (bn_layer_list): ModuleList(
        (0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (

========starting epoch 100==========
100-0, train loss 0.30070820450782776
train loss 0.30070820450782776
val metric {'loss': 0.2941181123256683, 'loss_no_reg': 0.2887372374534607, 'corr': [0.07352977748565663, 0.3811018813473136, 0.3520577203636432, 0.39981510367635287, 0.5791615920631378, 0.34135180898475437, 0.33221141054807635, 0.3935964147239117, 0.2831746662724414, 0.11054512771068796, 0.39837875182113797, 0.1370861272883418, 0.34500693280112604, 0.2416060430807726, 0.3535946803993614, 0.16204168773708963, 0.1808962524893817, 0.1321784091506123, 0.10500250236115698, 0.21279174346469898, 0.2220435384494003, 0.35590177381481597, 0.11712833585863666, 0.2961904080378846, 0.20571036923226146, 0.1320560812065854, 0.11410519460260163, 0.4907988418826686, 0.33462458602906464, 0.30013530594158166, 0.23120493184170454, 0.42471128700828076, 0.5349432824554505, 0.23812645248138203, 0.33386457070542375, 0.44413263233141154, 0.11246128240038868, 0.49737754129441786, 0.2826940780822861, 0.04288

========starting epoch 300==========
300-0, train loss 0.2764189541339874
train loss 0.2764189541339874
val metric {'loss': 0.2750251442193985, 'loss_no_reg': 0.26727405190467834, 'corr': [0.23111257071316416, 0.44644572007492467, 0.41821690281939, 0.44237683776371084, 0.5743599079667427, 0.45314026616227415, 0.3726407337608895, 0.40544406478834805, 0.2984002546400698, 0.1522970087683591, 0.4551578979415099, 0.16032560497434586, 0.4730173998644853, 0.23925862193593145, 0.4873703416918177, 0.17462482632419696, 0.2471674757145974, 0.11749208768045373, 0.23097334397138305, 0.4509510979507269, 0.3796921308916002, 0.5079739488207883, 0.32493949928830795, 0.3702229534327693, 0.40860338916268024, 0.22644818391684607, 0.41595618991386585, 0.5091686462952156, 0.5869266071784124, 0.4063863269997249, 0.347228707742129, 0.44395076919155385, 0.6057457757919283, 0.4253175762132737, 0.3974647018153592, 0.49839321050313595, 0.2590742325154642, 0.5307419686513625, 0.4638674368140465, 0.0917607133134008

========starting epoch 500==========
500-0, train loss 0.24851959943771362
train loss 0.24851959943771362
val metric {'loss': 0.26748629510402677, 'loss_no_reg': 0.2595166563987732, 'corr': [0.2275702618681873, 0.47768319682385, 0.41702517708280423, 0.4481771088006814, 0.6070233497861612, 0.4785857071495804, 0.3867166108342255, 0.40237026792204367, 0.29608113306915135, 0.15102881293387702, 0.461729551286943, 0.16450380429267333, 0.5346240853130839, 0.25205221471440453, 0.5025243421605152, 0.17431590367344116, 0.2833087328957693, 0.10890155297430357, 0.2570600561527926, 0.5115957109984761, 0.40573260212898293, 0.5499348137185384, 0.34961972905147604, 0.400127970565628, 0.47072992961564863, 0.3909243607162256, 0.44414739853892954, 0.5096583257947754, 0.6021688397028154, 0.40751237013510444, 0.3448016259367966, 0.42709261484026173, 0.6144746426178224, 0.437485261372285, 0.4393329824971095, 0.4948195863784373, 0.26152547040423146, 0.5353074632937962, 0.5156740154490499, 0.10765723447304015

========starting epoch 700==========
700-0, train loss 0.25937291979789734
train loss 0.25937291979789734
val metric {'loss': 0.26404690444469453, 'loss_no_reg': 0.2560061514377594, 'corr': [0.21834686713541748, 0.4837841774367565, 0.42425903484790023, 0.4513272347733669, 0.6111861099021649, 0.4729798550041288, 0.39960057630671775, 0.40424430495141656, 0.2928051059683793, 0.1599528237131769, 0.4701047063125687, 0.19029171384091778, 0.5342727829211436, 0.2539011570876885, 0.5012074992164625, 0.18675770993298724, 0.2903608495064066, 0.17676295240942808, 0.26617455229896175, 0.5144303920865381, 0.4348930397176336, 0.5629179137168552, 0.35092215102012336, 0.40520563427874934, 0.5003827727340785, 0.44792876467850873, 0.4399763085295531, 0.5179707213908806, 0.6045674596026095, 0.40557439415168983, 0.3576158963381751, 0.42859564084669166, 0.6169225558530872, 0.4410022179714946, 0.44479683706065903, 0.5039488019917402, 0.27818635951307785, 0.542521760850924, 0.5390161689936367, 0.1092755654251

...

========starting epoch 500==========
500-0, train loss 0.23885618150234222
train loss 0.23885618150234222
val metric {'loss': 0.26266120076179506, 'loss_no_reg': 0.2548428475856781, 'corr': [0.23514778117143634, 0.4840004577628173, 0.4199700781501948, 0.45574232447238483, 0.5965650105797023, 0.44235881925945797, 0.39319648095912285, 0.41796650796643386, 0.27374231215261136, 0.16028183193183843, 0.4805225106939598, 0.21052429030832778, 0.5457608816528634, 0.2773945095173317, 0.5123953077518326, 0.17768485625617667, 0.2939648819689644, 0.25355459059625274, 0.2221120535679841, 0.4970432294492205, 0.4332722170416648, 0.577999896317544, 0.35510631788595726, 0.41877712930384603, 0.5052254581547511, 0.45053321934541474, 0.4572823045292571, 0.5194934871944856, 0.5962816766968019, 0.4169484664973528, 0.38233467362716417, 0.43747306755917664, 0.6062856812224793, 0.43672503061682333, 0.44383633676242545, 0.5017003467910881, 0.2911600165343996, 0.5388364354149053, 0.5451461497263965, 0.11778036503

========starting epoch 100==========
100-0, train loss 0.2303396463394165
train loss 0.2303396463394165
val metric {'loss': 0.26162215769290925, 'loss_no_reg': 0.2536889910697937, 'corr': [0.23714432866366675, 0.48505711746914804, 0.4248369133015218, 0.45573895805802755, 0.6019489150095503, 0.46399628113509417, 0.39881482555688946, 0.4174068672164165, 0.27950209606917203, 0.15904872090041638, 0.47726651461045577, 0.21060136251684877, 0.5446359642737967, 0.27682362371441327, 0.5140855623324928, 0.17526785302053985, 0.29633232844748486, 0.24953638271202994, 0.23123329101487133, 0.5079612721465661, 0.4355867795403256, 0.5751167271435247, 0.35554150120413225, 0.4180189695938364, 0.504685292369763, 0.4550920970526156, 0.4552169152995411, 0.5215815153796576, 0.5996825097011762, 0.41438330246083566, 0.37687586633765285, 0.4408147025106698, 0.6066773878391314, 0.4373518376702043, 0.44255998529614127, 0.504284955548806, 0.2918382556092762, 0.5405185956278206, 0.5486305372060833, 0.1182539762499

========starting epoch 300==========
300-0, train loss 0.2377917468547821
train loss 0.2377917468547821
val metric {'loss': 0.2619974732398987, 'loss_no_reg': 0.2541056275367737, 'corr': [0.2366442423846331, 0.48405766038307485, 0.4229614798465568, 0.4546967477350136, 0.6001694003257325, 0.4605995524258923, 0.3993219757491716, 0.41705671413812384, 0.2784508454455988, 0.16006757632904153, 0.4783787765512635, 0.21152480148466993, 0.544628090624244, 0.277823400046886, 0.5150140474170446, 0.17491769829017328, 0.2953544047438003, 0.2520251333931383, 0.23165809396207582, 0.5050555943953016, 0.4361392191689474, 0.5773331769616925, 0.3552977496340056, 0.41823910145582477, 0.5042980180848824, 0.4501441591414533, 0.45670253570994795, 0.5216817414281965, 0.5982660299902527, 0.41588603914148087, 0.3783027600198027, 0.44039915553077974, 0.6082746515848266, 0.4358105926201161, 0.4445433109461281, 0.5033793124349103, 0.2902068460947286, 0.5403131689116814, 0.548092212384813, 0.11734876331144796, 0.38

========starting epoch 500==========
500-0, train loss 0.26022714376449585
train loss 0.26022714376449585
val metric {'loss': 0.2620244026184082, 'loss_no_reg': 0.25406867265701294, 'corr': [0.2369008757162004, 0.48365205451844967, 0.4228876506248005, 0.4557517341378521, 0.601346016857262, 0.4595862132089544, 0.39827357419098, 0.41726324569765044, 0.2783917594940024, 0.1590279590601848, 0.4787750112004875, 0.21200599015461952, 0.5449391385778771, 0.27821918592835443, 0.5148943820172154, 0.1754774980644254, 0.2950686678026884, 0.2527000668917474, 0.22707447480758114, 0.5037484345241477, 0.4350717685788964, 0.5741354034113485, 0.35555890757216424, 0.4183466852742568, 0.5030656167929896, 0.452486648085735, 0.4562667233919014, 0.5207513560045706, 0.5978797765747886, 0.4149156524933793, 0.37940274279512815, 0.44063736307219276, 0.6101624574824717, 0.43565944531485307, 0.44237564664414986, 0.5014551304002552, 0.2909464886114778, 0.5405349835115054, 0.5473283036076255, 0.11864910508533022, 0.

0.5071455326246751
